In [ ]:
import requests
import pandas as pd

# API base URL and parameters
api_url = "https://npiregistry.cms.hhs.gov/api/"
params = {
    "version": "2.1",
    "enumeration_type": "NPI-2",
    "state": "CO",
    "taxonomy_description": "hospice",
    "limit": 200,  # max allowed per call
    "skip": 0
}

all_results = []

while True:
    response = requests.get(api_url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break
    
    data = response.json()
    results = data.get("results", [])
    
    if not results:
        break
    
    for entry in results:
        npi = entry.get("number", "")
        basic = entry.get("basic", {})
        name = basic.get("organization_name", "")
        status = basic.get("status", "")
        
        # Practice locations can be multiple; take the first one
        addresses = entry.get("addresses", [])
        practice_address = next((addr for addr in addresses if addr.get("address_purpose") == "LOCATION"), {})
        full_address = f"{practice_address.get('address_1', '')} {practice_address.get('city', '')}, {practice_address.get('state', '')} {practice_address.get('postal_code', '')}"
        
        # Get taxonomy entries
        taxonomies = entry.get("taxonomies", [])
        taxonomy_entries = [(t.get("desc", ""), t.get("primary")) for t in taxonomies]
        
        # Get issuer list
        identifiers = entry.get("identifiers", [])
        issuers = [iden.get("issuer", "") for iden in identifiers if iden.get("issuer")]

        all_results.append({
            "NPI": npi,
            "Name": name,
            "Status": status,
            "Primary Practice Address": full_address,
            "Taxonomy Entries": taxonomy_entries,
            "Issuers": issuers
        })

    # Pagination: skip next batch
    params["skip"] += 200
    if len(results) < 200:
        break  # Stop if fewer than 200 returned (i.e., last page)

# Convert to DataFrame
df = pd.DataFrame(all_results)


In [13]:
df.to_csv("hospice_organizations_CO.csv", index=False)

In [5]:
df

,NPI,Name,Status,Primary Practice Address,Taxonomy Entries,Issuers
0,1760093470,247 HOME HEALTH CARE LTD,A,"8055 E TUFTS AVE STE 250 DENVER, CO 802372857","[(Hospice Care, Community Based, True)]",[]
1,1740072065,A BETTER COLORADO HOSPICE LLC,A,"126 W D ST STE 200 PUEBLO, CO 810034430","[(Hospice Care, Community Based, True)]",[]
2,1003483330,"A PEACEFUL JOURNEY HOSPICE, LLC",A,"2851 S PARKER RD STE 1130 AURORA, CO 800142732","[(Hospice Care, Community Based, True)]",[]
3,1861097982,"ABODE HEALTHCARE COLORADO, INC",A,"1050 EAGLERIDGE BLVD PUEBLO, CO 810082130","[(Hospice Care, Community Based, True)]",[]
4,1326459025,"ABODE HEALTHCARE COLORADO, INC",A,"5465 MARK DABLING BLVD COLORADO SPRINGS, CO 80...","[(Hospice Care, Community Based, True)]",[]
...,...,...,...,...,...,...
297,1194839001,WYOMING HOME HEALTH INC,A,"1103 E BOXELDER RD STE JB GILLETTE, WY 827185557","[(Case Management, False), (Day Training, Deve...",[]
298,1013648583,YNA HOSPICE INC,A,"3190 S VAUGHN WAY STE 550 OFF 520 AURORA, CO 8...","[(Hospice Care, Community Based, True)]",[]
299,1285645382,YULIYA GOSTISHCHEVA,A,"1240 S PARKER RD DENVER, CO 802317558","[(Hospice Care, Community Based, True)]",[]
300,1942931415,ZA HOSPICE INC,A,102 S TEJON ST STE 1100 OFF 1111 COLORADO SPRI...,"[(Hospice Care, Community Based, True)]",[]


In [6]:
# ZIP9 (may have zip9 or zip5)
df["ZIP9"] = df["Primary Practice Address"].str.extract(r'(\d{5}(?:-\d{4})?)$')

In [7]:
# 5 digit zip (ZIP5)
df['ZIP5'] = df['ZIP9'].str[:5]

df['ZIP5']

0      72857
1      34430
2      42732
3      82130
4      83842
       ...  
297    85557
298    80014
299    17558
300    80903
301    42735
Name: ZIP5, Length: 302, dtype: object

In [ ]:
# Hospice facilities status - all Active
df["Status"].value_counts()

Status
A    302
Name: count, dtype: int64

In [ ]:
# Are there any duplicate NPIs? - NO
df[df["NPI"].duplicated(keep=False)]

,NPI,Name,Status,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5


In [ ]:
# Are there any duplicate Names? YES 117 are duplicate names
df[df["Name"].duplicated(keep=False)]

,NPI,Name,Status,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5
3,1861097982,"ABODE HEALTHCARE COLORADO, INC",A,"1050 EAGLERIDGE BLVD PUEBLO, CO 810082130","[(Hospice Care, Community Based, True)]",[],82130,82130
4,1326459025,"ABODE HEALTHCARE COLORADO, INC",A,"5465 MARK DABLING BLVD COLORADO SPRINGS, CO 80...","[(Hospice Care, Community Based, True)]",[],83842,83842
5,1447611512,"ABODE HEALTHCARE COLORADO, INC.",A,"744 HORIZON CT STE 110 GRAND JUNCTION, CO 8150...","[(Hospice Care, Community Based, True)]",[],63915,63915
6,1760169221,"ABODE HEALTHCARE COLORADO, INC.",A,"200 E 7TH ST STE 200B LOVELAND, CO 805374864","[(Hospice Care, Community Based, True)]",[],74864,74864
7,1275919029,"ABODE HEALTHCARE COLORADO, INC.",A,"445 UNION BLVD STE 223 LAKEWOOD, CO 802281241","[(Hospice Care, Community Based, True)]",[],81241,81241
...,...,...,...,...,...,...,...,...
283,1689365033,VALLEY VIEW HOSPITAL ASSOCIATION,A,"1830 BLAKE AVE STE 202 GLENWOOD SPRINGS, CO 81...","[(Internal Medicine, Hospice and Palliative Me...",[],14261,14261
290,1811753023,WELLNESS PLUG LLC,A,"1562 S PARKER RD STE 320C DENVER, CO 802312721","[(Hospice Care, Community Based, True)]",[],12721,12721
291,1699592584,WELLNESS PLUG LLC,A,"1562 S PARKER RD STE 320C DENVER, CO 802312721","[(Home Health, True), (Hospice Care, Community...",[Colorado Department of Public Health and Envi...,12721,12721
296,1346699725,WYOMING HOME HEALTH INC,A,"1103 E BOXELDER RD STE JB GILLETTE, WY 827185557","[(Case Management, False), (Home Health, False...",[],85557,85557


In [ ]:
# Are there any duplicate Names AND Address? YES - 39
df[df.duplicated(subset=["Name", "Primary Practice Address"], keep=False)]

,NPI,Name,Status,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5
7,1275919029,"ABODE HEALTHCARE COLORADO, INC.",A,"445 UNION BLVD STE 223 LAKEWOOD, CO 802281241","[(Hospice Care, Community Based, True)]",[],81241,81241
8,1427469113,"ABODE HEALTHCARE COLORADO, INC.",A,"445 UNION BLVD STE 223 LAKEWOOD, CO 802281241","[(Hospice Care, Community Based, True)]",[],81241,81241
45,1861205908,BANNER HEALTH,A,"2001 70TH AVE STE 100 GREELEY, CO 806344628","[(Hospice Care, Community Based, True)]",[],44628,44628
46,1407409964,BANNER HEALTH,A,"2001 70TH AVE STE 100 GREELEY, CO 806344628","[(Hospice Care, Community Based, True)]",[],44628,44628
62,1639280548,CATHOLIC HEALTH INITIATIVES COLORADO,A,"9100 E MINERAL CIR CENTENNIAL, CO 801123401","[(Hospice Care, Community Based, True)]",[],23401,23401
64,1457731929,CATHOLIC HEALTH INITIATIVES COLORADO,A,"9100 E MINERAL CIR CENTENNIAL, CO 801123401","[(Internal Medicine, Hospice and Palliative Me...",[],23401,23401
79,1326043019,COLORADO VISITING NURSE ASSOCIATION,A,"6750 W 52ND AVE ARVADA, CO 800023928","[(Hospice Care, Community Based, True)]",[],23928,23928
80,1861004228,COLORADO VISITING NURSE ASSOCIATION,A,"6750 W 52ND AVE ARVADA, CO 800023928","[(Family Medicine, Hospice and Palliative Medi...",[],23928,23928
118,1851990980,HOPEWEST,A,"3090 N 12TH ST UNIT B GRAND JUNCTION, CO 81506...","[(Family Medicine, Hospice and Palliative Medi...",[],62804,62804
119,1326646886,HOPEWEST,A,"195 STAFFORD LN DELTA, CO 814162229","[(Family Medicine, Hospice and Palliative Medi...",[],62229,62229


In [ ]:
def contains_hospice(entry_list):
    for entry in entry_list:
        for field in entry:
            if isinstance(field, str) and "hospice" in field.lower():
                return True
    return False

# Apply the function
has_hospice = df["Taxonomy Entries"].apply(contains_hospice)

# Summary
print("Total NPIs:", len(df))
print("NPIs with at least one 'Hospice' taxonomy:", has_hospice.sum())

if has_hospice.all():
    print("All NPIs have at least one taxonomy with 'Hospice'.")
else:
    print("Some NPIs are missing a 'Hospice' taxonomy.")
    display(df[~has_hospice][["NPI", "Name", "Taxonomy Entries"]])


Total NPIs: 302
NPIs with at least one 'Hospice' taxonomy: 302
All NPIs have at least one taxonomy with 'Hospice'.


In [17]:
# removing duplicates by name and Primary Care Address
df = df.drop_duplicates(subset=["Name", "Primary Practice Address"], keep="first")

In [ ]:
df #279 total

,NPI,Name,Status,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5
0,1760093470,247 HOME HEALTH CARE LTD,A,"8055 E TUFTS AVE STE 250 DENVER, CO 802372857","[(Hospice Care, Community Based, True)]",[],72857,72857
1,1740072065,A BETTER COLORADO HOSPICE LLC,A,"126 W D ST STE 200 PUEBLO, CO 810034430","[(Hospice Care, Community Based, True)]",[],34430,34430
2,1003483330,"A PEACEFUL JOURNEY HOSPICE, LLC",A,"2851 S PARKER RD STE 1130 AURORA, CO 800142732","[(Hospice Care, Community Based, True)]",[],42732,42732
3,1861097982,"ABODE HEALTHCARE COLORADO, INC",A,"1050 EAGLERIDGE BLVD PUEBLO, CO 810082130","[(Hospice Care, Community Based, True)]",[],82130,82130
4,1326459025,"ABODE HEALTHCARE COLORADO, INC",A,"5465 MARK DABLING BLVD COLORADO SPRINGS, CO 80...","[(Hospice Care, Community Based, True)]",[],83842,83842
...,...,...,...,...,...,...,...,...
296,1346699725,WYOMING HOME HEALTH INC,A,"1103 E BOXELDER RD STE JB GILLETTE, WY 827185557","[(Case Management, False), (Home Health, False...",[],85557,85557
298,1013648583,YNA HOSPICE INC,A,"3190 S VAUGHN WAY STE 550 OFF 520 AURORA, CO 8...","[(Hospice Care, Community Based, True)]",[],80014,80014
299,1285645382,YULIYA GOSTISHCHEVA,A,"1240 S PARKER RD DENVER, CO 802317558","[(Hospice Care, Community Based, True)]",[],17558,17558
300,1942931415,ZA HOSPICE INC,A,102 S TEJON ST STE 1100 OFF 1111 COLORADO SPRI...,"[(Hospice Care, Community Based, True)]",[],80903,80903


In [20]:
df["Issuers"].value_counts()

Issuers
[]                                                                 266
[NPI, LICENSE]                                                       1
[State ID]                                                           1
[State of Colorado Department of Public Health and Environment]      1
[License Number]                                                     1
[Nebraska State Licensure]                                           1
[State Hospice Licensure]                                            1
[CO State License Number]                                            1
[Colorado license]                                                   1
[License]                                                            1
[Provider Number]                                                    1
[MEDICARE]                                                           1
[BCBS, HMO of CO, Medicare B]                                        1
[Colorado State License]                                             

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 0 to 301
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   NPI                       279 non-null    object
 1   Name                      279 non-null    object
 2   Status                    279 non-null    object
 3   Primary Practice Address  279 non-null    object
 4   Taxonomy Entries          279 non-null    object
 5   Issuers                   279 non-null    object
 6   ZIP9                      279 non-null    object
 7   ZIP5                      279 non-null    object
dtypes: object(8)
memory usage: 19.6+ KB


In [23]:
df.to_pickle('df.pkl')